# Working with huge datasets

There are a lot of ways to do this, this is a hard-working, understandable one that doesn't require extra packages.

In [1]:
!pwd

/c/Users/Domhnall/Projects/class_col_foundations/10-classwork


In [2]:
!ls

10-open-311-data.ipynb
10-scratchpad.txt
311_ServiceRequests_2010toPresent.csv
bbc-data.csv
bbc-data-2019-06-22-12-00.csv
root@138.197.73.93
scraper.py
sources


## Step 1: Download 311 data if you don't have it

**WARNING:** This is a 12gb file!

In [ ]:
# import os
# if not os.path.exists("311_ServiceRequests_2010toPresent.csv"):
#     !wget -o 311_ServiceRequests_2010toPresent.csv https://nycopendata.socrata.com/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD

## How many lines is it?

In [3]:
!wc -l 311_ServiceRequests_2010toPresent.csv

20979907 311_ServiceRequests_2010toPresent.csv


## Let's read in the file and check it out

It's very big, though, so we'll only look at the first 10 rows

In [2]:
import pandas as pd

pd.set_option("display.max_columns", 100)
%matplotlib inline

In [3]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=5)
df.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,37788938,11/27/2017 07:25:45 PM,11/29/2017 08:30:44 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11216,717 STERLING PLACE,STERLING PLACE,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,11/29/2017 08:30:44 AM,08 BROOKLYN,3012380047,BROOKLYN,996698,184444,PHONE,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.672924,-73.955125,"(40.67292417080954, -73.95512510845282)"
1,37788939,11/27/2017 12:26:36 AM,11/29/2017 02:06:34 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11208,72 NICHOLS AVENUE,NICHOLS AVENUE,NaN,NaN,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,11/29/2017 02:06:34 AM,05 BROOKLYN,3041090048,BROOKLYN,1020371,190356,ONLINE,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.689086,-73.869753,"(40.68908631153338, -73.8697526207326)"
2,37788940,11/27/2017 07:16:26 PM,11/28/2017 08:34:50 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10022,226 EAST 59 STREET,EAST 59 STREET,NaN,NaN,NaN,NaN,ADDRESS,NEW YORK,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,11/28/2017 08:34:50 PM,06 MANHATTAN,1013320035,MANHATTAN,993717,216613,PHONE,Unspecified,MANHATTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.761224,-73.965826,"(40.76122403578927, -73.96582633456184)"
3,37788942,11/27/2017 12:38:35 PM,12/04/2017 09:25:27 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11693,92-15 DE SOTA ROAD,DE SOTA ROAD,NaN,NaN,NaN,NaN,ADDRESS,Far Rockaway,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,12/04/2017 09:25:27 AM,14 QUEENS,4161270008,QUEENS,1035315,153169,ONLINE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.586943,-73.816148,"(40.58694256699123, -73.81614817308076)"
4,37788943,11/27/2017 04:38:14 AM,12/04/2017 02:07:07 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10456,1239 BOSTON ROAD,BOSTON ROAD,NaN,NaN,NaN,NaN,ADDRESS,BRONX,NaN,NaN,Closed,NaN,The Department of Housing Preservation and Dev...,12/04/2017 02:07:07 AM,03 BRONX,2026150050,BRONX,1011490,241747,PHONE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.830173,-73.901568,"(40.83017310705689, -73.90156802986618)"


# How much space does it take up?

Again, we won't read it all in, but let's see how much space 100,000 rows takes up.

In [4]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000)
df.memory_usage().sum()

C:\Program Files (x86)\Python37-32\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (17,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


19600042

Let's save that as our baseline. How much smaller can we make it?

In [5]:
original_size = df.memory_usage().sum()

## Read it all in as strings

Strings take up a lot of space, generally. How big is it if we say everything is a string?

In [6]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000, dtype='str')
df.memory_usage().sum()

16400042

In [7]:
df.memory_usage().sum() / original_size * 100

83.67350437310287

## Specify the data type

Since strings are big, maybe we can specify a different data type.

* `str` is big
* `float` and `int` are small
* `category` looks like strings, but is great for things that are actually categories (even things like cities!) - It's secretly stored as an integer, but written out to us as a string.

I made you a dictionary to play around with!

In [28]:
# What can be an int or float?
# What can be a category?

dtypes = {
    'Unique Key': 'str',
    'Created Date': 'str',
    'Closed Date': 'str', 
    'Agency': 'category',
    'Agency Name': 'category',
    'Complaint Type': 'category',
    'Descriptor': 'category',
    'Location Type': 'category',
    'Incident Zip': 'category',
    'Incident Address': 'str', 
    'Street Name': 'str',
    'Cross Street 1': 'str',
    'Cross Street 2': 'str',
    'Intersection Street 1': 'str',
    'Intersection Street 2': 'str',
    'Address Type': 'str',
    'City': 'category',
    'Landmark': 'category',
    'Facility Type': 'str',
    'Status': 'category',
    'Due Date': 'str',
    'Resolution Description': 'str',
    'Resolution Action Updated Date': 'str',
    'Community Board': 'category',
    'BBL': 'str',
    'Borough': 'category',
    'X Coordinate (State Plane)': 'float',
    'Y Coordinate (State Plane)': 'float',
    'Open Data Channel Type': 'str',
    'Park Facility Name': 'str', 
    'Park Borough': 'category',
    'Vehicle Type': 'category',
    'Taxi Company Borough': 'category',
    'Taxi Pick Up Location': 'str',
    'Bridge Highway Name': 'str',
    'Bridge Highway Direction': 'str',
    'Road Ramp': 'str',
    'Bridge Highway Segment': 'str',
    'Latitude': 'float',
    'Longitude': 'float',
    'Location': 'str'
}

In [29]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000, dtype=dtypes)
df.memory_usage().sum()

14131130

In [30]:
df.memory_usage().sum() / original_size * 100

72.09744754628588

## Only read in some columns

The easiest way to more easily read in data is to only keep a few columns!

In [12]:
columns = ['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Incident Zip',
       'Incident Address', 'Address Type',
       'Community Board', 'Borough',
       'Latitude', 'Longitude']
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000, usecols=columns)

In [13]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000, usecols=columns)
df.memory_usage().sum()

/Users/jonathansoma/.pyenv/versions/3.6.7/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


11200080

In [14]:
df.memory_usage().sum() / original_size * 100

34.14650208170224

# Only read in some columns and specify the data type

If you can combine the two techniques - reading in a subset of columns *and* specifying the data type - you can read in a lot more data without taking up nearly as much RAM.

In [31]:
dtypes = {
    'Unique Key': 'str',
    'Created Date': 'str',
    'Closed Date': 'str',
    'Agency': 'str',
    'Agency Name': 'str',
    'Complaint Type': 'str',
    'Descriptor': 'str',
    'Incident Zip': 'str',
    'Incident Address': 'str',
    'Address Type': 'str',
    'Community Board': 'str',
    'Borough': 'str',
    'Latitude': 'str',
    'Longitude': 'str'
}
columns = dtypes.keys()

In [32]:
df = pd.read_csv("311_ServiceRequests_2010toPresent.csv", nrows=100000, usecols=columns, dtype=dtypes)
df.memory_usage().sum()

5600042

In [33]:
df.memory_usage().sum() / original_size * 100

28.57158163232507